In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE






In [25]:
tda = pd.read_csv('C:\\Users\\Eric\\Downloads\\decrypted_file\\train\\train_data_ads.csv')#your directory
tdf =  pd.read_csv('C:\\Users\\Eric\\Downloads\\decrypted_file\\train\\train_data_feeds.csv')

In [26]:
tdf.rename(columns={'u_userId': 'user_id'}, inplace=True)  #change u_userId to user_id for better merging algotithm
tda.rename(columns={'user_id': 'user_id'}, inplace=True)  

tda = tda.sample(frac= 0.01)#remove this row in real process
tdf = tdf.sample(frac= 0.01)#remove this row in real process

In [27]:
tda['pt_d'] = tda['pt_d'].astype(str)

# Extract the month and time
tda['month'] = tda['pt_d'].str[4:6]#only consider month
tda['day'] = tda['pt_d'].str[6:8]#only consider date
tda['hour']= tda['pt_d'].str[8:10]#only consider hour

tda.drop(columns=['pt_d'], inplace=True)





In [28]:
tda['hour']

5720680    11
1088930    08
2810415    01
1448645    09
2741800    02
           ..
6957884    12
5995458    07
6546373    11
2809815    03
1918365    06
Name: hour, Length: 76755, dtype: object

In [29]:
tda.drop(["city","device_size"], axis=1, inplace= True)    #city is too specific, keep state instead of city should be good
tdf.drop(['u_browserMode','e_pl','e_et',"i_s_sourceId", "i_entities"], axis=1, inplace= True)#some duplicated information and not useful information(please double check for me)

In [30]:
#merge data by user_id
merged_data = pd.merge(tda, tdf, on='user_id', how='outer')#merge them by user_id

# create a feature indicating if the user clicked on the ad (i.e., userID exists in both datasets)
merged_data['clicked'] = merged_data.apply(lambda row: row['user_id'] in tda['user_id'].values and row['user_id'] in tdf['user_id'].values, axis=1)
merged_data.fillna(0, inplace=True )

merged_data.drop(['user_id','log_id','task_id','adv_id','i_docId','device_name','series_dev','slot_id','spread_app_id','site_id'], axis = 1, inplace= True)# some data that I would consider to drop based on their distribution. They are either too biased or not meaningful. like ID type of variables. I also delete the column with too many unique value

In [31]:
#sparse classes combination for the value that exist less than 4 times
value_frequencies = merged_data["residence"].value_counts()
sparse_classes = value_frequencies[value_frequencies < 4].index
merged_data["residence_combined"] = merged_data["residence"].apply(lambda x: 'Other' if x in sparse_classes else x)


In [32]:
merged_data

,label_x,age,gender,residence,city_rank,series_group,emui_dev,net_type,creat_type_cd,adv_prim_id,...,e_ch,e_m,e_po,e_rn,e_section,label_y,cillabel,pro,clicked,residence_combined
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.0,369.0,7.0,2.0,0.0,-1.0,-1.0,0.0,False,0.0
1,0.0,5.0,2.0,46.0,2.0,8.0,32.0,7.0,8.0,1036.0,...,19.0,508.0,7.0,9.0,0.0,-1.0,-1.0,0.0,True,46.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,1361.0,3.0,1.0,0.0,-1.0,-1.0,0.0,False,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.0,1361.0,2.0,2.0,0.0,1.0,-1.0,40.0,False,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.0,1361.0,12.0,4.0,0.0,-1.0,-1.0,0.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123653,1.0,7.0,2.0,39.0,5.0,3.0,28.0,4.0,3.0,1690.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,39.0
123654,0.0,7.0,2.0,39.0,5.0,3.0,28.0,6.0,5.0,1236.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,39.0
123655,0.0,3.0,2.0,41.0,5.0,7.0,21.0,7.0,10.0,1830.0,...,19.0,196.0,5.0,4.0,0.0,-1.0,-1.0,0.0,True,41.0
123656,0.0,8.0,2.0,41.0,5.0,3.0,20.0,7.0,8.0,1535.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,41.0


In [33]:

# Split the column into multiple columns(like 23151^231^214^123, I separate this long list to multiple column. I think the numerical number could represent certain category, and after ^ is the subset of this category)

def div_cols(df, col_name):
    split_columns = df[col_name].astype(str).str.split('^', expand=True)
    split_columns.columns = [f'{col_name}_{i+1}' for i in range(split_columns.shape[1])]
    split_columns.fillna(0, inplace=True)

    df = pd.concat([df, split_columns], axis=1)
    df = df.loc[:, ~df.columns.duplicated()]  # drop duplicated columns
    df.drop(col_name, axis=1, inplace=True)
    return df

# Define the split_columns_with_delimiter function
def split_columns_with_delimiter(df, delimiter='^'):
    columns_to_split = [col for col in df.columns if df[col].astype(str).str.contains(delimiter).any()]
   
    for col in columns_to_split:
        df = div_cols(df, col)
   
    return df
merged_data = split_columns_with_delimiter(merged_data)


In [34]:
from sklearn.preprocessing import LabelEncoder

# Identify and encode categorical columns
categorical_cols = merged_data.select_dtypes(include=['object', 'category']).columns
label_encoder = LabelEncoder()

for col in categorical_cols:
    merged_data[col] = merged_data[col].astype(str)
    merged_data[col] = label_encoder.fit_transform(merged_data[col])


In [35]:
merged_data.drop(columns=['creat_type_cd_1','inter_type_cd_1'])

,label_x_1,age_1,gender_1,residence_1,city_rank_1,series_group_1,emui_dev_1,net_type_1,adv_prim_id_1,hispace_app_tags_1,...,e_ch_1,e_m_1,e_po_1,e_rn_1,e_section_1,label_y_1,cillabel_1,pro_1,clicked_1,residence_combined_1
0,0,0,0,0,0,0,0,0,0,0,...,10,128,22,12,0,0,0,0,0,0
1,0,4,1,35,1,7,19,6,16,31,...,10,146,22,70,0,0,0,0,1,35
2,0,0,0,0,0,0,0,0,0,0,...,2,65,18,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,10,65,12,12,0,2,0,3,0,0
4,0,0,0,0,0,0,0,0,0,0,...,10,65,4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123653,1,6,1,28,4,2,15,3,323,7,...,0,0,0,0,0,1,1,0,0,28
123654,0,6,1,28,4,2,15,5,115,7,...,0,0,0,0,0,1,1,0,0,28
123655,0,2,1,30,4,6,10,6,389,35,...,10,103,20,34,0,0,0,0,1,30
123656,0,7,1,30,4,2,9,6,250,31,...,0,0,0,0,0,1,1,0,0,30


In [36]:

pos_click = merged_data[merged_data['clicked_1'] == True]

# filter rows where clicked is False
neg_click = merged_data[merged_data['clicked_1'] == False]

# save to csv files
pos_click.to_csv('pos_click.csv', index=False)
neg_click.to_csv('neg_click.csv', index=False)

In [37]:
pos_click

,label_x_1,age_1,gender_1,residence_1,city_rank_1,series_group_1,emui_dev_1,net_type_1,creat_type_cd_1,adv_prim_id_1,...,e_ch_1,e_m_1,e_po_1,e_rn_1,e_section_1,label_y_1,cillabel_1,pro_1,clicked_1,residence_combined_1
1,0,4,1,35,1,7,19,6,8,16,...,10,146,22,70,0,0,0,0,1,35
7,0,4,1,17,4,2,9,6,8,250,...,10,65,9,1,0,2,0,0,1,17
8,0,4,1,17,4,2,9,6,8,137,...,10,65,9,1,0,2,0,0,1,17
9,0,4,1,17,4,2,9,6,8,147,...,10,65,9,1,0,2,0,0,1,17
10,0,4,1,22,4,6,8,6,1,436,...,10,142,21,1,1,0,0,0,1,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123643,0,7,1,3,1,2,15,3,8,93,...,10,144,4,23,0,0,0,0,1,3
123644,0,7,1,3,1,2,15,3,8,93,...,10,144,6,1,0,0,0,0,1,3
123645,0,7,1,3,1,2,15,3,8,93,...,10,144,23,1,0,0,0,0,1,3
123646,0,4,2,24,1,2,10,6,8,16,...,10,196,11,1,0,0,0,0,1,24
